In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import autokeras as ak
import numpy as np
import pandas as pd
import os

from IPython.display import display

pd.set_option('display.max_columns', None)

sim_df = pd.read_csv("./analysis/24samps_expanding_sims/expanding-counts-SIMOUT.csv", sep=" ")
sim_df.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,zeta,zeta_e,psi,t_s,omega,taus_mean,taus_std,taus_skewness,taus_kurtosis,taus_median,taus_iqr,epsilons_mean,epsilons_std,epsilons_skewness,epsilons_kurtosis,epsilons_median,epsilons_iqr,Ne_s_mean,Ne_s_std,Ne_s_skewness,Ne_s_kurtosis,Ne_s_median,Ne_s_iqr,pop0-[[23__1]],pop0-[[22__2]],pop0-[[21__3]],pop0-[[20__4]],pop0-[[19__5]],pop0-[[18__6]],pop0-[[17__7]],pop0-[[16__8]],pop0-[[15__9]],pop0-[[14_10]],pop0-[[13_11]],pop0-[[12_12]],pop1-[[23__1]],pop1-[[22__2]],pop1-[[21__3]],pop1-[[20__4]],pop1-[[19__5]],pop1-[[18__6]],pop1-[[17__7]],pop1-[[16__8]],pop1-[[15__9]],pop1-[[14_10]],pop1-[[13_11]],pop1-[[12_12]],pop2-[[23__1]],pop2-[[22__2]],pop2-[[21__3]],pop2-[[20__4]],pop2-[[19__5]],pop2-[[18__6]],pop2-[[17__7]],pop2-[[16__8]],pop2-[[15__9]],pop2-[[14_10]],pop2-[[13_11]],pop2-[[12_12]],pop3-[[23__1]],pop3-[[22__2]],pop3-[[21__3]],pop3-[[20__4]],pop3-[[19__5]],pop3-[[18__6]],pop3-[[17__7]],pop3-[[16__8]],pop3-[[15__9]],pop3-[[14_10]],pop3-[[13_11]],pop3-[[12_12]],pop4-[[23__1]],pop4-[[22__2]],pop4-[[21__3]],pop4-[[20__4]],pop4-[[19__5]],pop4-[[18__6]],pop4-[[17__7]],pop4-[[16__8]],pop4-[[15__9]],pop4-[[14_10]],pop4-[[13_11]],pop4-[[12_12]],pop5-[[23__1]],pop5-[[22__2]],pop5-[[21__3]],pop5-[[20__4]],pop5-[[19__5]],pop5-[[18__6]],pop5-[[17__7]],pop5-[[16__8]],pop5-[[15__9]],pop5-[[14_10]],pop5-[[13_11]],pop5-[[12_12]],pop6-[[23__1]],pop6-[[22__2]],pop6-[[21__3]],pop6-[[20__4]],pop6-[[19__5]],pop6-[[18__6]],pop6-[[17__7]],pop6-[[16__8]],pop6-[[15__9]],pop6-[[14_10]],pop6-[[13_11]],pop6-[[12_12]]
0,0.741,5,1,880748,0.000,489149.667,283439.908,0.565,-1.500,367481.0,330764.00,13.059,2.241,0.642,-1.500,11.884,2.549,461659.857,213780.908,1.006,0.146,417871.0,173645.5,2478,1922,1608,1447,1096,1288,1783,1727,1443,1112,1886,1107,2220,1813,1812,1700,2112,1711,2274,1640,1446,1653,1288,736,2932,1950,1687,2287,1768,1686,1872,947,1542,1727,1485,868,2413,1793,1875,1483,1693,1092,1334,1223,1718,1639,1305,1152,2710,2332,1513,1679,1388,1749,1713,1240,1313,1786,1315,938,1385,1090,834,820,796,825,1104,979,1012,576,906,576,2395,1658,1789,1729,1895,1738,1327,1660,1602,1317,1549,666
1,0.789,6,1,674261,0.013,667122.500,7138.500,0.000,-2.000,667122.5,7138.50,4.716,0.100,0.000,-2.000,4.716,0.100,374263.571,315180.265,0.381,-1.478,211976.0,525269.0,1998,1338,1089,1004,822,831,1154,815,871,772,748,359,1876,1637,1321,990,885,1031,1041,1099,753,953,938,486,2064,1373,1338,980,1015,1023,950,848,826,898,805,445,1861,1473,931,999,1011,932,879,811,934,1078,1083,334,2041,1361,1355,1124,888,704,933,828,731,928,923,443,2202,1283,1140,1039,1051,1141,1237,1010,1004,738,973,407,3353,2340,1831,1715,1392,1713,1404,1272,1334,1576,1402,653
2,0.353,2,1,794799,0.000,528104.667,296869.179,-0.325,-1.530,606744.0,517510.75,14.271,4.197,-1.003,-0.293,15.929,3.850,535986.143,359535.932,0.035,-1.633,442496.0,647934.0,6736,6593,7482,5116,6373,6199,5066,5050,5562,3795,5209,3810,8080,7297,7568,4363,6653,5422,4465,6413,4608,4560,5373,1815,1455,822,833,740,710,613,627,570,658,635,500,212,7881,7303,5877,6244,6453,5569,6058,6312,6396,4714,5316,2304,661,538,597,518,303,405,288,313,366,312,300,321,6819,4990,4758,4411,2607,3223,2740,3119,2482,1897,2802,1141,62,33,21,27,20,16,15,16,12,11,10,7
3,0.326,2,1,420918,0.000,400676.000,357168.736,0.433,-1.526,270019.5,616313.25,11.341,4.576,-0.217,-0.607,11.396,3.972,428947.143,256310.630,0.056,-1.008,422579.0,301855.5,2106,1406,1337,1834,1302,1368,1133,1172,981,1288,865,611,1989,1534,1489,1486,1471,1031,1192,1254,1122,770,1418,507,4777,4097,3951,3718,3413,2817,3267,3057,3107,3104,2945,1189,991,604,598,436,443,338,354,373,290,423,422,96,9834,7224,6239,5344,4518,3643,3053,3095,3056,3451,3293,1111,635,433,477,405,350,337,400,351,403,352,312,221,12639,8571,6934,5094,3960,3545,3827,2658,3382,3318,2760,1262
4,0.597,4,1,726689,0.000,565212.500,224268.238,-0.086,-1.685,576893.5,364229.00,9.316,4.133,0.599,-1.230,8.071,5.667,461759.857,211717.260,-0.456,-1.340,517409.0,355083.0,1847,1259,1390,1612,1332,1512,1312,1334,1115,1311,1125,451,1906,1171,1314,1641,1272

In [19]:
train_size = int(sim_df.shape[0] * 0.9)
# Get just the msfs cols
msfs_df = sim_df.iloc[:, 23:]
msfs_df

,pop0-[[23__1]],pop0-[[22__2]],pop0-[[21__3]],pop0-[[20__4]],pop0-[[19__5]],pop0-[[18__6]],pop0-[[17__7]],pop0-[[16__8]],pop0-[[15__9]],pop0-[[14_10]],pop0-[[13_11]],pop0-[[12_12]],pop1-[[23__1]],pop1-[[22__2]],pop1-[[21__3]],pop1-[[20__4]],pop1-[[19__5]],pop1-[[18__6]],pop1-[[17__7]],pop1-[[16__8]],pop1-[[15__9]],pop1-[[14_10]],pop1-[[13_11]],pop1-[[12_12]],pop2-[[23__1]],pop2-[[22__2]],pop2-[[21__3]],pop2-[[20__4]],pop2-[[19__5]],pop2-[[18__6]],pop2-[[17__7]],pop2-[[16__8]],pop2-[[15__9]],pop2-[[14_10]],pop2-[[13_11]],pop2-[[12_12]],pop3-[[23__1]],pop3-[[22__2]],pop3-[[21__3]],pop3-[[20__4]],pop3-[[19__5]],pop3-[[18__6]],pop3-[[17__7]],pop3-[[16__8]],pop3-[[15__9]],pop3-[[14_10]],pop3-[[13_11]],pop3-[[12_12]],pop4-[[23__1]],pop4-[[22__2]],pop4-[[21__3]],pop4-[[20__4]],pop4-[[19__5]],pop4-[[18__6]],pop4-[[17__7]],pop4-[[16__8]],pop4-[[15__9]],pop4-[[14_10]],pop4-[[13_11]],pop4-[[12_12]],pop5-[[23__1]],pop5-[[22__2]],pop5-[[21__3]],pop5-[[20__4]],pop5-[[19__5]],pop5-[[18__6]],pop5-[[17__7]],pop5-[[16__8]],pop5-[[15__9]],pop5-[[14_10]],pop5-[[13_11]],pop5-[[12_12]],pop6-[[23__1]],pop6-[[22__2]],pop6-[[21__3]],pop6-[[20__4]],pop6-[[19__5]],pop6-[[18__6]],pop6-[[17__7]],pop6-[[16__8]],pop6-[[15__9]],pop6-[[14_10]],pop6-[[13_11]],pop6-[[12_12]]
0,2478,1922,1608,1447,1096,1288,1783,1727,1443,1112,1886,1107,2220,1813,1812,1700,2112,1711,2274,1640,1446,1653,1288,736,2932,1950,1687,2287,1768,1686,1872,947,1542,1727,1485,868,2413,1793,1875,1483,1693,1092,1334,1223,1718,1639,1305,1152,2710,2332,1513,1679,1388,1749,1713,1240,1313,1786,1315,938,1385,1090,834,820,796,825,1104,979,1012,576,906,576,2395,1658,1789,1729,1895,1738,1327,1660,1602,1317,1549,666
1,1998,1338,1089,1004,822,831,1154,815,871,772,748,359,1876,1637,1321,990,885,1031,1041,1099,753,953,938,486,2064,1373,1338,980,1015,1023,950,848,826,898,805,445,1861,1473,931,999,1011,932,879,811,934,1078,1083,334,2041,1361,1355,1124,888,704,933,828,731,928,923,443,2202,1283,1140,1039,1051,1141,1237,1010,1004,738,973,407,3353,2340,1831,1715,1392,1713,1404,1272,1334,1576,1402,653
2,6736,6593,7482,5116,6373,6199,5066,5050,5562,3795,5209,3810,8080,7297,7568,4363,6653,5422,4465,6413,4608,4560,5373,1815,1455,822,833,740,710,613,627,570,658,635,500,212,7881,7303,5877,6244,6453,5569,6058,6312,6396,4714,5316,2304,661,538,597,518,303,405,288,313,366,312,300,321,6819,4990,4758,4411,2607,3223,2740,3119,2482,1897,2802,1141,62,33,21,27,20,16,15,16,12,11,10,7
3,2106,1406,1337,1834,1302,1368,1133,1172,981,1288,865,611,1989,1534,1489,1486,1471,1031,1192,1254,1122,770,1418,507,4777,4097,3951,3718,3413,2817,3267,3057,3107,3104,2945,1189,991,604,598,436,443,338,354,373,290,423,422,96,9834,7224,6239,5344,4518,3643,3053,3095,3056,3451,3293,1111,635,433,477,405,350,337,400,351,403,352,312,221,12639,8571,6934,5094,3960,3545,3827,2658,3382,3318,2760,1262
4,1847,1259,1390,1612,1332,1512,1312,1334,1115,1311,1125,451,1906,1171,1314,1641,1272,824,1090,879,1167,1117,1808,687,1546,1434,1337,1458,1177,1013,1291,785,933,1165,1307,601,1605,1707,923,1206,935,1137,898,944,1146,1039,1522,442,782,493,403,404,345,357,305,359,299,401,229,149,2936,2338,2023,1697,1994,2234,1987,1548,1633,1816,1692,980,2710,2027,1642,1560,1580,1437,1054,1468,1388,1090,1382,731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,5603,4281,3632,3792,4211,3205,3108,3435,3262,2751,3498,2125,5343,3875,4904,3318,2781,3227,4755,3043,3947,3472,3303,1295,5947,4816,4051,4216,4151,4076,2983,3122,3438,3017,3902,1830,5071,5200,3809,3597,3753,3564,3506,3765,3374,5056,2589,1400,5645,4229,4314,3534,4019,4376,3175,2744,3693,3334,3978,1584,5878,5365,3088,3746,3853,3390,3791,2535,3217,3637,3522,1153,3009,2379,1818,1770,1451,1485,1161,1292,1076,1092,1366,665
39996,3155,2214,252